## <font size="5"> Object Detection Inference Notebook using IceVision/FastAI </font> ##

In [ ]:
import torch
from icevision.all import *
from icevision.models import *

from fastai.basics import *
from fastai.data.all import *
from fastai.callback import *

torch.cuda.set_device(0)

INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /home/aarlova/.icevision/mmdetection_configs/mmdetection_configs-2.20.1/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17


In [ ]:
checkpoint_path = '/media/14TB/aarlova_ovarian/Obj_det_retrain/training_log/faster_rcnn_resnet101_fpn_2x_ColorNorm_30_10x'

### <font size="5"> Recreate model from scratch to be able to import just the .pth checkpoint </font> ###

In [ ]:
#### new code
data_dir = Path('/media/14TB/aarlova_ovarian/YOLO_data/10x_withEmpty/data')
train = [(line.split("\n",1)[0]) for line in open('/media/14TB/aarlova_ovarian/YOLO_data/10x_withEmpty/train.txt')]
val = [(line.split("\n ",1)[0]) for line in open('/media/14TB/aarlova_ovarian/YOLO_data/10x_withEmpty/val.txt')]
train = [str.strip('\n') for str in train]
val = [str.strip('\n') for str in val]

test =[(line.split("\n",1)[0]) for line in open('/media/14TB/aarlova_ovarian/YOLO_data/10x_withEmpty/test.txt')]
test = [str.strip('\n') for str in test]


print(len(train), len(val), len(test))

data_splitter = FixedSplitter([train,val,test])

class_map = ClassMap(['Follicle'])

parser = parsers.VOCBBoxParser(annotations_dir=data_dir,
                     images_dir=data_dir,
                     class_map=class_map)

train_records, valid_records, test_records = parser.parse(data_splitter)

presize = 512
size = 512
train_tfms = tfms.A.Adapter([tfms.A.Resize(size, size), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([tfms.A.Resize(size, size), tfms.A.Normalize()])

train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

len(train_ds), len(valid_ds)


# model_type = models.ultralytics.yolov5
# backbone = model_type.backbones.small(pretrained=True)
# backbone = model_type.backbones.medium(pretrained=True)
# backbone = model_type.backbones.large(pretrained=True)
#backbone = model_type.backbones.extra_large(pretrained=True)
# model = model_type.model(backbone=backbone, num_classes=parser.class_map.num_classes, img_size=size, device=torch.device("cuda"))

# model_type = models.torchvision.retinanet
# backbone = model_type.backbones.resnet50_fpn

model_type = models.mmdet.faster_rcnn

backbone = model_type.backbones.resnet101_fpn_2x
# Instantiate the mdoel
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(parser.class_map))


train_dl = model_type.train_dl(train_ds, batch_size=16, num_workers=2, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=16, num_workers=2, shuffle=False)

metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

numeph = 30
magn = '10x'


model_name = 'faster_rcnn' + '_' + 'resnet101_fpn_2x' + '_' + 'ColorNorm_' + str(numeph) + '_' + magn
print(model_name)
csv_dir = '/media/14TB/aarlova_ovarian/Obj_det_retrain/'


# model_name = 'yolov5_small'

learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)



1143 657 439


  0%|          | 0/2558 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/2558 [00:00<?, ?it/s]

  0%|          | 0/243518604 [00:00<?, ?B/s]

/home/aarlova/anaconda3/envs/pytorch_lym_clone/lib/python3.9/site-packages/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  warnings.warn(
2022-12-29 11:18:13,199 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet101'}
2022-12-29 11:18:13,201 - mmcv - INFO - load model from: torchvision://resnet101
2022-12-29 11:18:13,202 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet101
2022-12-29 11:18:13,358 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-12-29 11:18:13,412 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-12-29 11:18:13,454 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-12-29 11:18:13,465 - mmcv - INFO -

load checkpoint from local path: checkpoints/faster_rcnn/faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([4]).
faster_rcnn_resnet101_fpn_2x_ColorNorm_30_10x


In [ ]:
learn.load(checkpoint_path)

/home/aarlova/anaconda3/envs/pytorch_lym_clone/lib/python3.9/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


### <font size="5"> Set up test dataset and dataloader, run inference on etire test set (tile-wise) (optional) </font> ###

In [ ]:
infer_ds = Dataset(test_records, valid_tfms) # valid_tfms are appropriate for test dataset in this case
infer_dl = model_type.infer_dl(infer_ds, batch_size=4, shuffle=False)

In [ ]:
## run inference on test dataset (optional)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)

  0%|          | 0/110 [00:00<?, ?it/s]

/home/aarlova/anaconda3/envs/pytorch_lym_clone/lib/python3.9/site-packages/mmdet/core/anchor/anchor_generator.py:324: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/home/aarlova/anaconda3/envs/pytorch_lym_clone/lib/python3.9/site-packages/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  warnings.warn(


In [ ]:
plt.rcParams['figure.dpi'] = 100 # for high resolution figure
show_preds(preds=preds[:10])

### <font size="5">Inference with OpenSlide on Ovarian valid/train slides (one annotated section per slide)</font>
<font size="3"> Ground Truth Margin annotations are imported from .json file </font>

In [ ]:
import openslide
import cv2
from skimage import morphology
import cv2 as cv
from shapely.geometry import Polygon, MultiPolygon, box
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import random
import numpy as np
from shapely.strtree import STRtree
from shapely.geometry import mapping

import geojson

In [ ]:
def get_grid(polys, tile_size, ol, plot = False):
    slide_patches = []
    for i in range(len(polys)):
        margin_poly = Polygon(polys[i])
        minx, miny, maxx, maxy = margin_poly.bounds
        minx, miny, maxx, maxy = int(minx), int(miny), int(maxx), int(maxy)

        n_cols = int(np.ceil((maxx - minx) / (tile_size - ol)))
        n_rows = int(np.ceil((maxy - miny) / (tile_size - ol)))

        # all x's
        x_zero = int(margin_poly.bounds[0])
        xs = []
        xs.append(x_zero)
        for x in range(n_cols - 1):
            next_x = x_zero + (tile_size - ol)
            xs.append(next_x)
            x_zero = next_x

        # all y's
        y_zero = int(margin_poly.bounds[1])
        ys = []
        ys.append(y_zero)
        for y in range(n_rows - 1):
            next_y = y_zero + (tile_size - ol)
            ys.append(next_y)
            y_zero = next_y

        # assemble into rectangles
        patches = []
        for w in range(n_cols):
            x = xs[w]
            for z in range(n_rows):
                y = ys[z]
                rect = box(x, y, x + tile_size, y + tile_size, ccw=False)
                patches.append(rect)

        # find patches that intersect with tissue margin
        tree = STRtree(patches)
        query_geom = margin_poly
        intersects = [n for n in tree.query(query_geom) if n.intersects(query_geom) and n.intersection(query_geom).area/n.area > 0.05]

        print('number of patches in this section: ', len(intersects))

        # append intersecting patches to slide_patches list
        slide_patches.append(intersects)

        if plot == True:
            # let's plot the intersecting ones
            fig, ax = plt.subplots()

            # plot main_poly
            ab = margin_poly.exterior.xy
            a, b = np.array(ab[0]), np.array(ab[1])
            ax.plot(a, b, color='b', linestyle='-', linewidth=0.4)
            ax.invert_yaxis()

            # plot other_polies
            patches = intersects
            color = ['r', 'g', 'b', 'c', 'y', 'm']
            for j in range(len(patches)):
                x, y = np.array(patches[j].exterior.xy[0]), np.array(patches[j].exterior.xy[1])
                plt.plot(x, y, color=color[random.randint(0, 5)], linestyle='-', linewidth=0.4)
            ax.grid(linestyle='--', linewidth='0.5')
            plt.title("Finding Intersecting Polys")
            plt.xlabel("X")
            plt.ylabel("Y")
            plt.axis('equal')
            plt.show()
            plt.cla()
            plt.clf()
            plt.close()

    return slide_patches

In [ ]:
def read_qupath_json(json_path):
    with open(json_path) as f:
        slide_objects = geojson.load(f)
        # slide_areas = [shape(obj["geometry"]).area for obj in slide_objects]
        slide_labels = [obj['properties']['classification']['name'] for obj in slide_objects]
        slide_object_coordinates = [obj['geometry']['coordinates'][0] for obj in slide_objects]
        return slide_object_coordinates, slide_labels

In [ ]:
def write_qupath_noIDs_Polys(xmlsave, regions, region_labels, region_colors):
    dumped = []
    trythis = '['
    for i in range(0, len(regions)):
        # if len(regions[i]) >2:
            # roi = Polygon(regions[i])
        roi = regions[i]
        label = region_labels[i]
        trythis += json.dumps(
            {"type": "Feature", "id": "PathAnnotationObject", "geometry": mapping(roi),
             "properties": {"classification": {"name": label, "colorRGB": region_colors}, "isLocked": False,
                            "measurements": []}}, indent=4)
        if i < len(regions) - 1:
            trythis += ','
        else:
            dumped.append([region_labels[i], regions[i]])
    trythis += ']'

    with open(xmlsave, 'w') as outfile:
        outfile.write(trythis)

        dumped = []

In [ ]:
from pathlib import Path

# slide_list = ['17118416']#, '17118483', '17118873', '17118922', '17119598', '17122090', '17122178', '17122189', '17122204', '17122207', '17122313', '17124767', '17126247', '17126281', '17126430', '17126465', '17126513', '17133040', '17133223', '17133268', '17133434', '17133461', '17133587', '17133626', '17133630', '17133678', '17133780', '17133781', '17133792', '17133937', '17133968', '17134444', '17134545', '17134750']
# slide_list = ['17133945','17122241','17121975', '17133388', '17133503', '17133754', '17134585'] # val
# slide_list = ['17121963', '17124810', '17122321', '17133557']#, '17133893'] # test
slide_list = ['17133893']
region_colors = -16274801
json_dir = Path('/media/14TB/aarlova_ovarian/ovarian demo/demo_objDet_preds')

In [ ]:
requested_magn = 10
downsample = int(40/requested_magn)
tile_size = int(500*downsample)
overlap = int(250*downsample)

In [ ]:
for i in range(len(slide_list)):
    slide_fname = Path('/media/14TB/aarlova_ovarian/slides/' + slide_list[i] + '.svs')
    print('Working on Slide', slide_fname.name)

    slide = openslide.OpenSlide(str(slide_fname))
    print('opened slide!')

    # get regions within which to perform inference
    jsons_fname = Path('/media/14TB/aarlova_ovarian/GT_json_annotations/' + slide_list[i] + '.json')

    coords, labels = read_qupath_json(jsons_fname)

    tissue_regions = [Polygon(coords[i]) for i in range(len(coords)) if labels[i] == 'Margin']
    print('There are ',len(tissue_regions),'annotated margins')

    # this is to fix one invalid area - delete this line
    tissue_regions[1] = tissue_regions[1].buffer(0)

    #######################################################################
    # generate a grid of tiles of specific size and overlap within bounds of the Ovary
    print('Set tile size to', tile_size, ', and overlap to', overlap)

    slide_grid = get_grid(tissue_regions, tile_size, overlap, plot=False) # slide_grid is a nested list of Polygons. Len(slide_grid) is how many sections of tissue are on slide.
    print('Done tiling the slide!')

    #######################################################################
    # tile the slide with my script, get slide_grid
    print('Number of sections in slide:', len(slide_grid))

    slide_boxes = []

    for section in range(len(slide_grid)):
        print('Number of tiles in current section:', len(slide_grid[section]))


        for tile in range(len(slide_grid[section])):
            current_tile = slide_grid[section][tile]
            current_tile_top_left_coord = [int(current_tile.bounds[0]), int(current_tile.bounds[1])]  # this should be minx, miny if polygons were created with ccw=False
            # tile_name = str(out_tile_dir/slide_fname.stem) + '/' + str(slide_fname.stem) + ' [x=' + str(current_tile_top_left_coord[0]) + ',y=' + str(current_tile_top_left_coord[1]) + ',w='+str(tile_size) + ',h=' + str(tile_size) + '].jpg'
            tile_img = slide.read_region((current_tile_top_left_coord[0], current_tile_top_left_coord[1]),0,(tile_size,tile_size)).convert('RGB')
            resized_tile = tile_img.resize((500,500))


            #2. Predict on one individual tile at a time
            pred_dict  = model_type.end2end_detect(resized_tile, valid_tfms, model, class_map=class_map, detection_threshold=0.25)
            # pred_dict['img'].show()

            poly_boxes = []

            a = pred_dict['detection']['bboxes']
            if len(a) > 0:
                for b in a:
                    poly = box(b.xmin, b.ymin, b.xmax, b.ymax)
                    poly_boxes.append(poly)



            #3. Translate tile-relative coords of Polygon to WSI-relative coords
            for p in range(len(poly_boxes)):
                new_ext_x = np.array(poly_boxes[p].exterior.coords.xy[0]) * downsample + current_tile_top_left_coord[0]
                new_ext_y = np.array(poly_boxes[p].exterior.coords.xy[1]) * downsample + current_tile_top_left_coord[1]

                new_coords = list(zip(new_ext_x, new_ext_y))
                new_poly = Polygon(new_coords)
                slide_boxes.append(new_poly)


            tile_img.close()
            resized_tile.close()



        print('done processing tiles in section', section)



    print('done with all sections')

    slide.close()

    ########### filter out 'too' rectangular polygons
    filtered = []
    for o in slide_boxes:
        x = o.bounds[2] - o.bounds[0]
        y = o.bounds[3] - o.bounds[1]

        abs_log_aspect_ratio_sample = abs(math.log(x/y))

        if abs_log_aspect_ratio_sample <= 0.5:
            filtered.append(o)

    ########## remove intersecting whose intersection with other is greater than 50% of its area?
    tree = STRtree(filtered)

    print('After filtered by aspect ratio', len(filtered))
    iou = []
    all_merged = []
    for i in filtered:
        query_geom = i
        # intersects = [o for o in tree.query(query_geom)
        #               if o.intersection(query_geom).area/unary_union([o,query_geom]).area > 0.5]
        intersects = [o for o in tree.query(query_geom) if o.intersection(query_geom).area/o.area > 0.5]

        merged = unary_union(intersects)
        all_merged.append(merged)

    print('After merging intersecting polygons', len(all_merged))

    ############### remove duplicates from all_merged:
    all_merged_coords = [f.bounds for f in all_merged] # convert to list of coords in order to find unique polygons
    unique_merged = set(all_merged_coords)
    print('Len of unique merged polygons after removing duplicates',len(unique_merged))
    unique_merged = [box(*u) for u in unique_merged] # back to Polygons

    ############### remove polygons that are covered by other polygons
    tree = STRtree(unique_merged)
    to_remove = []
    for i in unique_merged:
        query_geom = i
        covered_by = [o for o in tree.query(query_geom) if o.covered_by(query_geom) and not o.equals(query_geom)]
        to_remove.extend(covered_by) # list of polygons to remove

    ############### remove duplicates
    all_merged_coords = [f.bounds for f in unique_merged]
    to_remove_coords = [f.bounds for f in to_remove]
    unique_merged = set(all_merged_coords) - set(to_remove_coords)
    print('Len of unique merged after removing polygons that are covered by others',len(unique_merged))

    unique_merged = [box(*u) for u in unique_merged] # back to Polygons

    ############## write JSON files
    slide_follicles = unique_merged
    slide_labels = ['Follicle' for f in slide_follicles]

    write_qupath_noIDs_Polys(str(json_dir) + '/' + str(slide_fname.stem) + '_fastRCNN.json', slide_follicles, slide_labels, region_colors)
    # write all original preds
    slide_labels = ['Follicle' for f in slide_boxes]
    write_qupath_noIDs_Polys(str(json_dir) + '/' + str(slide_fname.stem) + '_fastRCNN_all.json', slide_boxes, slide_labels, region_colors)


Working on Slide 17133893.svs
opened slide!
There are  3 annotated margins
Set tile size to 2000 , and overlap to 1000
number of patches in this section:  65
number of patches in this section:  76
number of patches in this section:  248
Done tiling the slide!
Number of sections in slide: 3
Number of tiles in current section: 65
done processing tiles in section 0
Number of tiles in current section: 76
done processing tiles in section 1
Number of tiles in current section: 248
done processing tiles in section 2
done with all sections
After filtered by aspect ratio 959
After merging intersecting polygons 959
Len of unique merged polygons after removing duplicates 639
Len of unique merged after removing polygons that are covered by others 481
